In [1]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *

In [2]:
no_of_reviews = 10000

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
driver = webdriver.Chrome("/Users/reejungkim/Downloads/chromedriver")

wait = WebDriverWait( driver, 10 )

In [3]:
#Append your app store urls here
urls = ["https://play.google.com/store/apps/details?id=com.kakaobank.channel&hl=ko&showAllReviews=true"]

In [4]:
import pandas as pd
df = pd.DataFrame()

In [5]:
%%time

for url in urls:

    driver.get(url)

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")

    # open all reviews
    url = url+'&showAllReviews=true'
    driver.get(url)
    time.sleep(5) # wait dom ready
    for i in range(1,10):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')#scroll to load other reviews
        time.sleep(1)
    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")
    expand_pages = soup_expatistan.findAll("div", class_="d15Mdf")
    counter = 1
    for expand_page in expand_pages:
        try:
            
            author = str(expand_page.find("span", class_="X43Kjb").text)
            review_date = expand_page.find("span", class_="p2TkOb").text
            
            reviewer_ratings = expand_page.find("div", class_="jUL89d y92BAb").text
            #print("Reviewer Ratings: ", reviewer_ratings)

            review_body = str(expand_page.find("div", class_="UD7Dzf").text)
            #print("Review Body: ", review_body)
            
            star = expand_page.find("div", class_="pf5lIe").find_next()['aria-label']
            
            developer_reply = expand_page.find_parent().find("div", class_="LVQB0b")

            new_row = {'author':author, 'review_date':review_date, 
                       'reviewer_ratings':reviewer_ratings, 'review':review_body, 'star':star,
                       'developer_reply': developer_reply
                }
            df = df.append(new_row, ignore_index=True)

            counter+=1
        except:
            pass
driver.quit()

CPU times: user 1.12 s, sys: 30.1 ms, total: 1.15 s
Wall time: 20.1 s


In [6]:
reviewer_ratings

'38'

In [7]:
df.head() 

,author,developer_reply,review,review_date,reviewer_ratings,star
0,풀메탈,None,저는 잘 사용하고 있어요. 근데 건의할게 있어요. 이체할때 실시간 잔금이 나오면 ...,2021년 2월 21일,9,별표 5개 만점에 4개를 받았습니다.
1,김영후,None,"자주쓰는 계좌 등록이 없어서 매달.새로작성. 관리비,세금. 국세 자동이체는 전혀안...",2021년 2월 22일,7,별표 5개 만점에 2개를 받았습니다.
2,혜지,None,모바일 앱 로그인이 되지않아 고객센터 안내 받아 앱 삭제후 재설치하는데 재설치 조...,2021년 2월 7일,80,별표 5개 만점에 1개를 받았습니다.
3,PrADᅡ KᅵllᄐR,None,"계좌송금시, 금액부터 적는게 아니라 받을사람 계좌정보부터 쓰고 확인한 후에 보낼 ...",2021년 2월 14일,56,별표 5개 만점에 3개를 받았습니다.
4,김수현,None,갤럭시 s10 사용자입니다 갤럭시 업데이트 이후 아무것도 안 되네요ㅠㅠ 리뷰 보니...,2021년 2월 23일,0,별표 5개 만점에 1개를 받았습니다.


In [8]:
import datetime as dt
timestamp = dt.datetime.today().strftime('%Y-%m-%d-%H:%m:%S')

In [9]:
file_name = 'GooglePlayStore_Review_' + timestamp + '.csv'

In [10]:
df.to_csv(file_name, index = True)

driver.quit()